In [ ]:
# !pip install pandas

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd drive/MyDrive/chinese_text_classification/
!pwd

/content/drive/MyDrive/chinese_text_classification
/content/drive/MyDrive/chinese_text_classification


In [48]:
import pandas as pd
df = pd.read_csv('./data.csv', encoding='gbk')
df.head()

,建言类型,问题,改进意见
0,安全隐患,用户隐私数据保护措施不完善,加强数据加密和权限申请，确保用户个人信息安全。
1,安全隐患,存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。
2,安全隐患,系统登录认证方式薄弱，易受到非法访问,改进系统登录认证机制，增加多重身份验证，提高账号安全性。
3,安全隐患,缺乏日志监控和异常检测机制,建立日志监控和异常检测系统，及时发现和应对安全威胁。
4,安全隐患,内部员工权限管理混乱，存在安全隐患,加强员工权限管理，实行权限分级制度，防止数据泄露。


In [49]:
df.rename(columns={'建言类型':'label', '问题':'text', '改进意见':'suggestions'}, inplace=True)
# df.head()
df['label'] = df['label'].apply(lambda x: str(x))
df['text'] = df['text'].apply(lambda x: str(x))
df['suggestions'] = df['suggestions'].apply(lambda x: str(x))
df['text1'] = df['text'] + ',' + df['suggestions']

df.head()


,label,text,suggestions,text1
0,安全隐患,用户隐私数据保护措施不完善,加强数据加密和权限申请，确保用户个人信息安全。,"用户隐私数据保护措施不完善,加强数据加密和权限申请，确保用户个人信息安全。"
1,安全隐患,存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。,"存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。"
2,安全隐患,系统登录认证方式薄弱，易受到非法访问,改进系统登录认证机制，增加多重身份验证，提高账号安全性。,"系统登录认证方式薄弱，易受到非法访问,改进系统登录认证机制，增加多重身份验证，提高账号安全性。"
3,安全隐患,缺乏日志监控和异常检测机制,建立日志监控和异常检测系统，及时发现和应对安全威胁。,"缺乏日志监控和异常检测机制,建立日志监控和异常检测系统，及时发现和应对安全威胁。"
4,安全隐患,内部员工权限管理混乱，存在安全隐患,加强员工权限管理，实行权限分级制度，防止数据泄露。,"内部员工权限管理混乱，存在安全隐患,加强员工权限管理，实行权限分级制度，防止数据泄露。"


In [50]:
temp = df['text1']
df.drop(labels=['text'], axis=1, inplace=True)
df.drop(labels=['suggestions'], axis=1, inplace=True)
df.rename(columns={'text1':'text'}, inplace=True)
df.head()

,label,text
0,安全隐患,"用户隐私数据保护措施不完善,加强数据加密和权限申请，确保用户个人信息安全。"
1,安全隐患,"存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。"
2,安全隐患,"系统登录认证方式薄弱，易受到非法访问,改进系统登录认证机制，增加多重身份验证，提高账号安全性。"
3,安全隐患,"缺乏日志监控和异常检测机制,建立日志监控和异常检测系统，及时发现和应对安全威胁。"
4,安全隐患,"内部员工权限管理混乱，存在安全隐患,加强员工权限管理，实行权限分级制度，防止数据泄露。"


### 将标签从汉字准换为数字

In [65]:
df['label'] = df['label'].map({'安全隐患':0, '客户服务':1, '市场运营':2, '网络运营':3})

In [ ]:
# !pip install pip install scikit-learn



In [66]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'],stratify=df['label'])

## 预训练模型微调

In [67]:
from transformers import AutoModel, BertTokenizerFast
bert = AutoModel.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

In [70]:
train_idx = x_train.dropna().index
test_idx = x_test.dropna().index

train_tokens = tokenizer.batch_encode_plus(x_train[train_idx].to_list(),
                      max_length = 50,
                      pad_to_max_length = True,
                      truncation = True)
test_tokens = tokenizer.batch_encode_plus(x_test[test_idx].to_list(),
                      max_length = 50,
                      pad_to_max_length = True,
                      truncation = True)


In [71]:
import torch
import numpy as np
train_seq = torch.tensor(train_tokens['input_ids'])
train_mask = torch.tensor(train_tokens['attention_mask'])
train_y = torch.tensor(y_train[train_idx].to_list())

test_seq = torch.tensor(test_tokens['input_ids'])
test_mask = torch.tensor(test_tokens['attention_mask'])
test_y = torch.tensor(y_test[test_idx].to_list())

In [72]:
from torch.utils.data import TensorDataset, RandomSampler, DataLoader

In [73]:
train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
trainloader = DataLoader(train_data,
              sampler = train_sampler,
              batch_size = 32)

test_data = TensorDataset(test_seq, test_mask, test_y)
test_sampler = RandomSampler(test_data)
testloader = DataLoader(test_data,
            sampler = test_sampler,
            batch_size = 32)

In [74]:
for param in bert.parameters():
    param.requires_grad = False

In [88]:
from torch import nn
from transformers import AdamW
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight

class BertClassifier(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.fc1 = nn.Linear(768,4)

    def forward(self, sent_id, mask):
        _ , cls_hs = self.bert(sent_id, attention_mask = mask, return_dict=False)
        return F.log_softmax(self.fc1(cls_hs), dim = 1)

In [89]:
model = BertClassifier(bert)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optimizer = AdamW(model.parameters(), lr = 1e-5)

class_weights = compute_class_weight(class_weight='balanced',
                    classes=np.unique(y_train),
                    y=y_train)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [90]:
weights = torch.tensor(class_weights, dtype = torch.float)
weights = weights.to(device)

criterion = nn.NLLLoss(weight = weights)

### 微调模型

In [ ]:
from tqdm.notebook import tqdm

epoches = 20

for e in range(epoches):
    train_loss = 0.0
    for batch in tqdm(trainloader):
        batch = [i.to(device) for i in batch]
        sent_id, masks, labels = batch

        optimizer.zero_grad()
        preds = model(sent_id, masks)
        loss = criterion(preds, labels)
        train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
    print(f'Epoch:{e+1}\t\tTraining Loss: {train_loss / len(trainloader)}')

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch:1		Training Loss: 1.4206649661064148


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch:2		Training Loss: 1.4150760968526204


  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch:7		Training Loss: 1.4088860352834065


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch:8		Training Loss: 1.3861205180486043


  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
pred_label = []
true_label = []
for batch in tqdm(testloader):
    batch = [i.cuda() for i in batch]
    sent_id, masks, labels = batch

    preds = model(sent_id, masks)
    pred_label.extend(torch.argmax(preds, axis = 1).cpu())
    true_label.extend(labels.cpu())

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(true_label, pred_label)
print(classification_report(true_label, pred_label))